In [10]:
# Building CNN and print out summary
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('minions') and 1 for the other ('spongebob')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [11]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        './images/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=44,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 44 images belonging to 2 classes.


In [17]:
# Triggering model training and pushing training performance telemetry data to Azure Machine Learning (AML)
run = experiment.start_logging()
epochs = 3

history = model.fit(
      train_generator,
      steps_per_epoch=1,  
      epochs=epochs,
      verbose=1)

#print(history.history)

def average_loss(lol):
    return sum(lol) / len(lol)
lol = history.history['loss']
averaged_loss = average_loss(lol)
#print(averaged_loss)

def average_accuracy(loa):
    return sum(loa) / len(loa)
loa = history.history['accuracy']
averaged_accuracy = average_loss(loa)
#print(averaged_accuracy)

run.log('epochs', np.float(epochs))
run.log('average_loss', np.float(averaged_loss))
run.log('average_accuracy', np.float(averaged_accuracy))

lol = history.history['loss']
for x in range(len(lol)):
    run.log('loss', np.float(lol[x])),
    #print(lol[x]), 
    
loa = history.history['accuracy']
for x in range(len(loa)):
    run.log('accuracy', np.float(loa[x])),
    #print(loa[x]),

#print(run.id)
run.complete()

  ...
    to  
  ['...']
Train for 1 steps
Epoch 1/3
1/1 [==============================] - 1s 1s/step - loss: 0.4567 - accuracy: 0.8409
Epoch 2/3
1/1 [==============================] - 1s 1s/step - loss: 0.2880 - accuracy: 0.9545
Epoch 3/3
1/1 [==============================] - 1s 1s/step - loss: 0.1734 - accuracy: 0.9545
